In [ ]:
!pip install datasets

##connect to huggingface_hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install huggingface_hub

import huggingface_hub
huggingface_hub.login(token="hf_token")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


##Load dataset

In [ ]:
from datasets import load_dataset, load_metric
common_voice_train = load_dataset("mozilla-foundation/common_voice_6_1", "fa", split="train")
common_voice_test = load_dataset("mozilla-foundation/common_voice_6_1", "fa", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


##Time filtering

In [ ]:
def filter_audio_files(dataset, min_length, max_length):
    def is_valid_length(example):
        # Calculate length in seconds
        audio_length = len(example["audio"]["array"]) / example["audio"]["sampling_rate"]
        return min_length <= audio_length <= max_length
    return dataset.filter(is_valid_length)

In [ ]:
common_voice_train_filtered = filter_audio_files(common_voice_train, min_length=4, max_length=6)
common_voice_test_filtered = filter_audio_files(common_voice_test, min_length=0, max_length=15)

In [ ]:
print(f"Number of training files after filtering: {len(common_voice_train_filtered)}")
print(f"Number of test files after filtering: {len(common_voice_test_filtered)}")

Number of training files after filtering: 2217
Number of test files after filtering: 5212


In [ ]:
common_voice_train = common_voice_train_filtered
common_voice_test = common_voice_test_filtered

In [ ]:
print(f"Number of training files after filtering: {len(common_voice_train)}")
print(f"Number of test files after filtering: {len(common_voice_test)}")

Number of training files after filtering: 2217
Number of test files after filtering: 5212


In [ ]:
print(common_voice_train)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 2217
})


##Remove redundant information from the dataset

In [ ]:
common_voice_train = common_voice_train.remove_columns(["down_votes", "gender", "locale", "segment", "up_votes" , "accent", "age", "client_id"])
common_voice_test = common_voice_test.remove_columns(["down_votes", "gender", "locale", "segment", "up_votes" , "accent", "age", "client_id"])

In [ ]:
print(common_voice_train)

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2217
})


##Data preprocessing

In [ ]:
!pip install hazm

In [ ]:
from hazm import Normalizer
chars_to_mapping = {
    'ى': 'ی', 'ك': 'ک', 'أ': 'ا', 'ؤ': 'و', 'إ': 'ا', 'ة': 'ه', 'ۀ': 'ه', 'ہ': 'ه',
    'ۂ': 'ه', 'ے': 'ی', 'ۓ': 'ی', 'ې': 'ی', 'ي': 'ی', 'آ': 'ا'
}

chars_to_ignore = [
    '!', '#', '�', '’', "'", '’', '%', '’’', ':', ';', '-', '!', '.', '?', ',', '؟', '؟', '-', '!', '.', '?', ',', 'ٔ', '٬', 'ٔ', '؛', '(', ')', '،', '«', '»',
    ';', ':', '”', '’‘', '%', '‘‘', '=', '–', '…', '_', '‘‘', '‘', '„', 'ā', 'š'
]

chars_to_ignore = set(chars_to_ignore)
normalizer = Normalizer()

In [ ]:
import re
from collections import defaultdict


In [ ]:
unique_chars = defaultdict(int)

In [ ]:
for i in range(len(common_voice_train)):
    text = common_voice_train[i]['sentence']
    text = normalizer.normalize(text)

    for src_char, dst_char in chars_to_mapping.items():
        text = text.replace(src_char, dst_char)

    text = ''.join(char for char in text if char not in chars_to_ignore)
    common_voice_train[i]['sentence'] = text
    for char in text:
        unique_chars[char] += 1

In [ ]:
for i in range(len(common_voice_test)):
    text = common_voice_test[i]['sentence']
    text = normalizer.normalize(text)

    for src_char, dst_char in chars_to_mapping.items():
        text = text.replace(src_char, dst_char)

    text = ''.join(char for char in text if char not in chars_to_ignore)
    common_voice_test[i]['sentence'] = text
    for char in text:
        unique_chars[char] += 1

In [ ]:
unique_chars['<s>'] = 1
unique_chars['</s>'] = 1
unique_chars['|'] = 1
unique_chars['<unk>'] = 1
vocab = sorted(unique_chars.keys())
print("تعداد حروف متمایز:", len(vocab))
print(vocab)


تعداد حروف متمایز: 64
[' ', '"', '&', '</s>', '<s>', '<unk>', 'A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'M', 'Q', 'S', 'T', 'U', 'e', 'm', 'n', 'o', 'u', 'y', '|', 'ء', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c', 'ﯾ', 'ﯿ']


In [ ]:
!pip install torchaudio librosa transformers

In [ ]:
import torchaudio
import librosa
import transformers
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
import os

##Making a dictionary

In [ ]:
import json
vocab_dict = {char: idx for idx, char in enumerate(vocab)}
with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False, indent=4)

##Resampling

In [ ]:
def resample_audio(example):
    audio = example["audio"]["array"]
    audio_resampled = librosa.resample(audio, orig_sr=48000, target_sr=16000)
    example["audio"]["array"] = audio_resampled
    example["audio"]["sampling_rate"] = 16000
    return example

common_voice_train = common_voice_train.map(resample_audio)
common_voice_test = common_voice_test.map(resample_audio)


Map:   0%|          | 0/2217 [00:00<?, ? examples/s]

Map:   0%|          | 0/5212 [00:00<?, ? examples/s]

##Tokenization and feature extraction

In [ ]:
tokenizer_save_path = "./wav2vec2-base-mine/"

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")
tokenizer.save_pretrained(tokenizer_save_path)

('./wav2vec2-base-mine/tokenizer_config.json',
 './wav2vec2-base-mine/special_tokens_map.json',
 './wav2vec2-base-mine/vocab.json',
 './wav2vec2-base-mine/added_tokens.json')

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor.save_pretrained(tokenizer_save_path)

['./wav2vec2-base-mine/preprocessor_config.json']

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained(tokenizer_save_path)

[]

In [ ]:
print(common_voice_train)

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2217
})


##Create Data Collator

In [ ]:
def speech_file_to_array_fn(batch):
    try:
        inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding=True)
        batch["input_values"] = inputs.input_values[0]
        batch["attention_mask"] = inputs.attention_mask[0]
        batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    except Exception as e:
        logger.error(f"Error processing batch {batch}: {e}")
    return batch

In [ ]:
common_voice_train_processed = common_voice_train.map(speech_file_to_array_fn, remove_columns=["audio", "sentence"])
common_voice_test_processed = common_voice_test.map(speech_file_to_array_fn, remove_columns=["audio", "sentence"])

Map:   0%|          | 0/2217 [00:00<?, ? examples/s]

Map:   0%|          | 0/5212 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import Wav2Vec2Processor
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install jiwer

In [ ]:
from datasets import load_metric

wer_metric = load_metric("wer")

<ipython-input-41-eaec034da022>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The repository for wer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


##Compute metrics

In [ ]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(pred.label_ids, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

##Definition and preparation of Wav2Vec2ForCTC model

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    attention_dropout=0.1,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab())
)
model.freeze_feature_extractor()


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2284: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch] -U

In [ ]:
!pip show accelerate
!pip show transformers

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Name: transformers
Version: 4.42.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./wav2vec2-large-xlsr-53-mine/",
    group_by_length=True,
    per_device_train_batch_size=6,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=250,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import TrainerCallback, TrainerState, TrainerControl
class SaveModelCallback(TrainerCallback):
    def on_epoch_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        output_dir = f"{args.output_dir}/checkpoint-{state.epoch}"
        kwargs["model"].save_pretrained(output_dir)
        kwargs["tokenizer"].save_pretrained(output_dir)
        print(f"Model saved at epoch {state.epoch} to {output_dir}")


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train_processed,
    eval_dataset=common_voice_test_processed,
    tokenizer=processor.feature_extractor,
    callbacks=[SaveModelCallback]
)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

accelerator.state._reset_state()

accelerator = Accelerator()

##Train model

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,13.340800,21.938522,0.999006


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Model saved at epoch 1.0 to ./wav2vec2-large-xlsr-53-mine//checkpoint-1.0


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Model saved at epoch 2.0 to ./wav2vec2-large-xlsr-53-mine//checkpoint-2.0


TrainOutput(global_step=740, training_loss=21.450221953520902, metrics={'train_runtime': 3023.8501, 'train_samples_per_second': 1.466, 'train_steps_per_second': 0.245, 'total_flos': 5.832002736862111e+17, 'train_loss': 21.450221953520902, 'epoch': 2.0})

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 94421697
Trainable parameters: 90221249
